In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb

***vis_completables_progress* function**:
* Input: players_info dictionary 
* Displays bar chart showing for each player the progress achieved in the different completables of the game as well as in the total game

In [ ]:
outputCompletableProgress=widgets.Output()
def vis_completables_progress():
    out=interactive(getProgressPlayerPerCompletable,
                      #constructing a dropdown to select how many data we want to see in visualisations for x axis
                      xaxis=widgets.Dropdown(options=[('All datas',0),('5 per visualisation',5),('25 per visualisation',25),('50 per visualisation',50),('10 first',-1),('10 last',-2)],
                                description='x axe',
                                disabled=False),
                      #constructing a dropdown to order for alphabetic keys or for values
                        order=widgets.Dropdown(options=[('alphabetic',0),('value from highest to lowest',1),('value from lowest to highest',-1)],
                                description='Order',
                                disabled=False))
    md_text=widgets.HTML("<h4>Bar chart with progress of players in each completable</h4>")
    with outputCompletableProgress:
        print("Updating visualisation with the current selection of players and completables...")
    return widgets.VBox([md_text,out, outputCompletableProgress])

def getProgressPlayerPerCompletable(xaxis, order):
    #get all players selected
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    completableSelectedProgress=getSublistSelectedFromUnionListSelected(completable_progress_list, checkboxesCompletableSelected.get_selected_options())
    completab={}
    for player in playersSelected:
        if not player in completab.keys():
            completab[player]={}
        if players_info[player]["completables_progress"] != {}:
            for c in completableSelectedProgress:
                if c in players_info[player]["completables_progress"].keys():
                    completab[player][c]=players_info[player]["completables_progress"][c]
                else:
                    completab[player][c]=0
    # progress of each player per completable
    players_progress = {}
    for player in completab.keys():
        for c in completableSelectedProgress:
            if not c in players_progress.keys():
                players_progress[c] = {}
            if c in completab[player].keys():
                players_progress[c][player] = completab[player][c]*100
            else:
                players_progress[c][player] = 0
    ordered_progress = orderDictByDictValues(players_progress,order)
    pos, bar_width = getPosDictMultiBarChart(ordered_progress,playersSelected)
    nbgraphshown, players_progress,N = organizeDictPerxAxisSelected(completableSelectedProgress,xaxis,players_progress)
    pos = organizeDictPosPerDataxaxis(xaxis,pos)
    for n in range(nbgraphshown):
        n_groups = len(players_progress[n])
        players_progress[n] = reorganiseDictBySubKeys(players_progress[n])
        if len(completableSelectedProgress)>0 and len(playersSelected)>0:
            constructMultiBarWithVerticalLines(players_progress,n,
                                            playersSelected, completableSelectedProgress,
                                            pos, bar_width, N)
        else:
            noDataToFillVis(100, True)
        plt.xlabel("Completable")
        plt.ylabel("Progress (in %)")
        plt.title("Progress of players per completable (in %)")
        plt.show()
        with outputCompletableProgress:
            clear_output(wait=False)